<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ufidon/nlp/blob/main/01.tned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ufidon/nlp/blob/main/01.tned.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>
<br>

# Text Normalization, Edit Distance

📝 SALP chapter 2

## What is Sentence Segmentation?
- **Sentence Segmentation** is the process of dividing a corpus (a large collection of text) into individual sentences.
  - Typically, sentences are separated by **punctuation marks** such as periods (.), exclamation marks (!), or question marks (?).
- Challenges
  - **Abbreviations:** e.g., "Dr.", "etc.", "Mr." could be mistaken as sentence boundaries.
  - **Quotes and Parentheses:** Sentences within quotes or parentheses can create ambiguity.
  - **Ellipses:** Ellipsis (...) can be mistaken as multiple sentence boundaries.

- 🍎 Segment text into sentences

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

text = "Mr. Smith went to Washington. He enjoyed his time there."
sentences = sent_tokenize(text)
print(sentences)

## Segmenting Sentences into Words
- **Word Tokenization** is the process of breaking down a sentence into individual words or tokens.
- Word tokenization helps in identifying the `basic units of meaning` in a text.
- Challenges
  - **Contractions:** e.g., "don't" can be split into "do" and "n't".
  - **Hyphenated Words:** e.g., "state-of-the-art" could be considered one word or multiple words.
- 🍎 segment a sentence into words

In [ ]:
from nltk.tokenize import word_tokenize

sentence = "Mr. Smith went to Washington."
words = word_tokenize(sentence)
print(words)

## Punctuation in Segmentation
- **Sentence Segmentation:** Periods, exclamation marks, and question marks usually denote the end of a sentence.
- **Word Tokenization:** Punctuation marks like commas, semicolons, and quotes are often separated from the words they are attached to.
- 🍎
  - Sentence: "Hello, world!"
  - Tokenized: ["Hello", ",", "world", "!"]
- Whether to keep or discard punctuation depends on the task. 
  - For sentiment analysis, punctuation can carry important emotional context.

## Spoken Language Components
- **Utterances:** A complete unit of speech that may not be grammatically correct.
- **Disfluencies:** Interruptions in the flow of speech, e.g., "uh," "um," "you know."
- **Fragments:** Incomplete sentences or phrases.
- **Fillers:** Words or sounds that fill gaps in speech but do not add meaning, e.g., "like," "you know."
- **Filled Pauses:** Prolonged sounds such as "uhhh" or "ummm."
- 🍎
  - Utterance: "Well, I... I think, um, it's great!"
  - Tokenized: ["Well", ",", "I", "...", "I", "think", ",", "um", ",", "it's", "great", "!"]
- Disfluencies may signal that the speaker is restarting the clause or idea
  - helpful in speech recognition in predicting the upcoming word
  - treated as regular words in such cases

## Word Types vs. Word Instances
- Word Types
  - **Word Types** refer to distinct words in a text, ignoring repetition.
  - 🍎: In the text "cat cat dog," "cat" and "dog" are the word types.
- Word Instances
  - **Word Instances** refer to the total occurrences of words in a text.
  - 🍎: In the text "cat cat dog," there are 3 word instances: two "cat" and one "dog."

## Case Sensitivity in Word Types
- **Case Sensitivity:** 
  - In some NLP tasks, "Word" and "word" may be considered different types.
  - This distinction is important in cases where capitalization changes meaning
    - e.g., "Apple" (company) vs. "apple" (fruit).
- **Normalization:** 
  - Often, texts are normalized to lowercase to treat "Word" and "word" as the same type for tasks like `word frequency analysis`.

- 🍎 case sensitivity

In [ ]:
words = ["Word", "word", "Apple", "apple"]
lowercase_words = [word.lower() for word in words]
print(set(lowercase_words)) 

| **Corpus Name**       | **Word Instances** | **Word Types**  | **Description**                                                                 |
|-----------------------|--------------------|-----------------|---------------------------------------------------------------------------------|
| **Brown Corpus**      | ~1,000,000         | ~50,000         | The first million-word electronic corpus of American English.                   |
| **Penn Treebank**     | ~4,500,000         | ~100,000        | Annotated corpus of English, primarily used for training NLP models.            |
| **British National Corpus (BNC)** | ~100,000,000      | ~600,000        | A wide range of modern British English (spoken and written).                    |
| **COCA (Corpus of Contemporary American English)** | ~1,000,000,000   | ~450,000        | Largest freely available corpus of American English, covering 1990-2019.        |
| **Google Ngrams**     | ~500,000,000,000   | ~13,588,391     | Dataset of n-grams (1 to 5 words) from scanned books dating from 1500-2008.     |
| **Reuters Corpus**    | ~810,000           | ~30,000         | Collection of news documents used for text classification and NLP research.     |
| **Wikipedia Corpus**  | ~1,000,000,000+    | ~20,000,000+    | Continuously updated corpus of Wikipedia articles, covering a vast range of topics. |
| **Gutenberg Corpus**  | ~25,000,000        | ~200,000        | Collection of classic literature and historical texts available as eBooks.      |

## 🏃 Explore [Google N-grams Corpus](https://books.google.com/ngrams/info) 
- an extensive dataset consisting of n-grams (sequences of 1 to 5 words) derived from a large corpus of books scanned by Google
- contain around 500 billion word instances, over 13 million unique word types
- [Google books syntactic N-grams](https://commondatastorage.googleapis.com/books/syntactic-ngrams/index.html)

## Herdan’s Law (Heaps’ Law)
- crucial for predicting vocabulary growth in linguistics and NLP
- as a text corpus grows, vocabulary size increases at a diminishing rate
- **Formal Definition:**
  - the vocabulary size $V$ grows as a sublinear power-law function of the number of word tokens $N$ in the corpus:
  - $ V(N) = k \times N^\beta $
    - $ V(N) $ is the number of distinct word types (vocabulary size).
    - $ N $ is the total number of word tokens.
    - $ k $ is a constant that depends on the corpus.
    - $ 0 < \beta <1 $ is an exponent that usually ranges between 0.4 and 0.6.
-  **Key Points:**
   - **Sublinear Growth**: 
     - As more words are added to the corpus, the rate of new word types (unique words) discovered decreases. 
     - Initially, the vocabulary grows quickly, but over time, as more text is added, the discovery of new unique words slows down.
   - **Vocabulary Saturation**: 
     - In any given language, there is a finite number of commonly used words. 
     - As the corpus size increases, fewer and fewer new words are encountered, and the vocabulary growth rate diminishes.
   - **Implications for Language Models**: 
     - Even with large corpora, the vocabulary size will not grow indefinitely and will eventually plateau.

## Wordform and Lemma
- A **wordform** is the specific written or spoken version of a word
  - including its tense, case, number, or mood, depending on the language.
  - 🍎
    - *Runs, running, ran* are different wordforms of the verb *run*.
    - *Cats, cat's, cats'* are different wordforms of the noun *cat*
- A **lemma** is the canonical or dictionary form of a word
  - representing all its possible wordforms.
  - the base or root form used to look up words in dictionaries.
  - often used in linguistic analysis to group different wordforms together.
  - 🍎
    - The lemma for the wordforms *runs, running, ran* is *run*.
    - The lemma for the wordforms *cats, cat's, cats'* is *cat*.
- The process of converting a wordform into its lemma is called **lemmatization**

## Unix Tools for Word Tokenization
- `tr` (translate): Convert or delete characters in text.
- `cut`: Remove sections from each line of files or input.
- `awk`: Pattern scanning and processing language.
- `grep`: Search for patterns in text.
  - 🍎 Tokenizing a text file
    ```bash
    grep -oE '\w+' input.txt > tokens.txt
    ```

In [ ]:
%%bash
# 1. `tr` replaces spaces with newlines
echo "Hello, world! How are you?" | tr -s ' ' '\n'

In [ ]:
%%bash
# 2. `cut` splits the text using a space as a delimiter and extracts the first word.
echo "Hello, world! How are you?" | cut -d ' ' -f 1

In [ ]:
%%bash
# 3. `awk` prints each word separately by iterating over fields (`$i`)
echo "Hello, world! How are you?" | awk '{for(i=1;i<=NF;i++) print $i}'

In [ ]:
%%bash
# 4. `grep` finds and prints each word matching the regex pattern `\w+` (alphanumeric characters).
echo "Hello, world! How are you?" | grep -oE '\w+'

In [ ]:
%%bash
# 5. Combining `tr` and `awk`
# - `tr -s ' ' '\n'` tokenizes by spaces.
# - `awk '{print tolower($0)}'` converts each token to lowercase.

echo "Hello, world! How are you?" | tr -s ' ' '\n' | awk '{print tolower($0)}'

## 🏃 Practice

In [ ]:
%%bash
# 0. Download [Shakespeare's](https://www.gutenberg.org/cache/epub/100/pg100.txt) works in a text file, save it as `sh.txt`
wget -O sh.txt https://www.gutenberg.org/cache/epub/100/pg100.txt 
# 1. ‘squeezed’ a series of non-alphabetic characters in a row into a single newline
tr -sc 'A-Za-z' '\n' < sh.txt # sh.txt contains all Shakespeare's works
# 2. find the vocabulary of Shakespeare's works
tr -sc 'A-Za-z' '\n' < sh.txt | sort | uniq -c
# 3. convert Shakespeare's vocabulary into lowercase
tr -sc 'A-Za-z' '\n' < sh.txt | tr A-Z a-z | sort | uniq -c
# 4. find frequent words
tr -sc 'A-Za-z' '\n' < sh.txt | tr A-Z a-z | sort | uniq -c | sort -n -r > corpus.txt


## Tokenization Methods in NLP
- Top-down (Rule-based) Tokenization
  - Uses predefined rules to split text into tokens
  - Typically faster but less flexible than machine learning approaches
  - Penn Treebank Tokenization
    - Developed for the [Penn Treebank Project](https://catalog.ldc.upenn.edu/LDC99T42)
    - Specific rules for English text

🍎

In [ ]:
from nltk.tokenize import word_tokenize

text = "Don't hesitate to email john.doe@example.com"
tokens = word_tokenize(text)
print(tokens)


### 🍎 NLTK RegexpTokenizer
- Uses regular expressions for flexible tokenization

In [ ]:
from nltk.tokenize import RegexpTokenizer

text = "Hello, world! How's it going?"
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)
print(tokens)

## Chinese, Korean, and Japanese Tokenization

- These languages don't use spaces between words
- Requires specialized algorithms
- Chinese Tokenization
  - Uses techniques like maximum matching or statistical models
- Korean Tokenization
  - Considers complex morphological structure
- Japanese Tokenization
  - Uses dictionary-based approaches or statistical models

In [ ]:
# Check if jieba is installed on Google Colab
import sys
in_colab = 'google.colab' in sys.modules
jieba_installed = 'jieba' in sys.modules

if in_colab and not jieba_installed:
    print("jieba is not installed. Installing now...")
    %pip install jieba

In [ ]:
# 🍎 1. Tokenize Chinese using jieba library

import jieba

text = "我喜欢自然语言处理"
tokens = jieba.cut(text)
print(list(tokens))

## Bottom-up Tokenization Algorithms
- also called Subword tokenization methods 
- efficiently handle rare words and out-of-vocabulary terms
- [Byte-Pair Encoding (BPE)](https://en.wikipedia.org/wiki/Byte_pair_encoding)
  - Merges the most frequent pair of bytes or characters until a desired vocabulary size is reached
  - 🍎 Word `"lower"` could be split into `"low"` + `"er"` if `"lower"` is rare but `"low"` and `"er"` are common.
- `Unigram Language Modeling (SentencePiece)`
  - Instead of merging, it starts with a large set of potential subword units.
  - Find the subset of subword units that maximizes the likelihood of the training data.

###  **Comparison: BPE vs. Unigram Language Modeling**
- **BPE**:
  - **Deterministic**: Iteratively merges most frequent pairs.
  - **Example**: `"lowered"` → `"low"` + `"ered"`.

- **Unigram**:
  - **Probabilistic**: Selects subwords to maximize likelihood.
  - **Example**: `"lowered"` → `"low"` + `"ered"` based on likelihood.

In [ ]:
# 🍎 1. BPE in Python

from tokenizers import Tokenizer, models, trainers

tokenizer = Tokenizer(models.BPE())
trainer = trainers.BpeTrainer(vocab_size=5000)
tokenizer.train(["corpus.txt"], trainer)
encoded = tokenizer.encode("lowered")
print(encoded.tokens)

In [ ]:
# 🍎 2. Unigram in Python

import sentencepiece as spm

spm.SentencePieceTrainer.train(input='corpus.txt', model_prefix='m', vocab_size=5000, model_type='unigram')
sp = spm.SentencePieceProcessor(model_file='m.model')
print(sp.encode_as_pieces("lowered"))



## ⚠️ In neural networks
  - Words are `NOT` used as the internal unit of representation at all! 
  - The input strings are tokenized into words even `only parts` of words.

## Byte-Pair Encoding (BPE)

- Initially developed as a text compression algorithm.
- Adopted by OpenAI for tokenization in the GPT models.
- used in GPT, GPT-2, RoBERTa, BART, DeBERTa.
- It iteratively merges `the most frequent pairs of symbols` in a sequence to reduce the overall number of symbols


### **Byte-Level BPE**

- **Purpose**: Handles all characters by using bytes instead of Unicode.
- **Base Vocabulary Size**: 256 (for all byte values).
- **Advantage**: Avoids unknown tokens for unseen characters.


### Steps of the BPE Algorithm:
1. **Initial Symbol Representation**: Treat each character (or byte) in the text as an individual symbol.
2. **Count Symbol Pairs**: Identify all pairs of consecutive symbols and count their occurrences.
3. **Merge the Most Frequent Pair**: Replace the most frequent pair of symbols with a new symbol (typically the concatenation of the two symbols).
4. **Repeat**: Repeat steps 2 and 3 until the desired number of merges is reached or no more frequent pairs exist.
5. **Tokenization**: After merging, tokenize the text using the final set of symbols.

### 🍎 Example:
Let's apply Byte Pair Encoding (BPE) to the following corpus:

```
low low lower new new newer better better best
```

### Step-by-Step BPE Tokenization on the Corpus

1. **Initial Corpus**:
    - We split the words into character sequences with an underscore `_` representing the end of each word. The initial vocabulary is:
    
    ```
    low:     'l o w _' (2 occurrences)
    lower:   'l o w e r _' (1 occurrence)
    new:     'n e w _' (2 occurrences)
    newer:   'n e w e r _' (1 occurrence)
    better:  'b e t t e r _' (2 occurrences)
    best:    'b e s t _' (1 occurrence)
    ```

    Initial vocabulary and frequencies:
    ```
    'l o w _'      -> 2
    'l o w e r _'  -> 1
    'n e w _'      -> 2
    'n e w e r _'  -> 1
    'b e t t e r _' -> 2
    'b e s t _'    -> 1
    ```

2. **Step 1: Count symbol pairs**:
    - Count the pairs of adjacent symbols in the vocabulary:
    ```
    ('l', 'o'):3, ('o', 'w'):3, ('w', '_'):4, ('r', '_'):4, ('e', 'r'):4, 
    ('w', 'e'):2, ('n', 'e'):3, ('e', 'w'):3, ('b', 'e'):3, ('t', 't'):2, ('t', 'e'):2, 
    ('e', 't'):2, ('t', '_'):1, ('e', 's'):1, ('s', 't'):1
    ```

    The first most frequent pair is `('w', '_')`.

3. **Step 2: Merge the most frequent pair**:
    - Replace all occurrences of `('w', '_')` with `w_`:
    ```
    Merge: ('w', '_'): 4
    Old: {'l o w _': 2, 'l o w e r _': 1, 'n e w _': 2, 'n e w e r _': 1, 'b e t t e r _': 2, 'b e s t _': 1}
    New: {'l o w_': 2, 'l o w e r _': 1, 'n e w_': 2, 'n e w e r _': 1, 'b e t t e r _': 2, 'b e s t _': 1}
    ```

4. **Step 3: Repeat the process**:

    **New symbol pairs**:
    ```
    ('l', 'o'):3, ('o', 'w_'):2, ('r', '_'):4, ('e', 'r'):4, ('o', 'w'):1, 
    ('w', 'e'):2, ('e', 'w_'):2, ('n', 'e'):3, ('e', 'w'):1, ('b', 'e'):3, ('t', 't'):2, 
    ('t', 'e'):2, ('e', 't'):2, ('t', '_'):1, ('e', 's'):1, ('s', 't'):1
    ```

    - The first most frequent pair is `('r', '_')`.

    **Merge `('lo', 'w')`**:
    ```
    'low _'      -> 2
    'low e r _'  -> 1
    'n e w _'    -> 2
    'n e w e r _' -> 1
    'b e t t e r _' -> 2
    'b e s t _'  -> 1
    ```

    **New symbol pairs**:
    ```
    ('low', '_'): 3
    ('w', 'e'): 1
    ('e', 'r'): 3
    ('r', '_'): 3
    ('n', 'e'): 3
    ('e', 'w'): 3
    ('b', 'e'): 3
    ('e', 't'): 2
    ('t', 't'): 2
    ('t', 'e'): 2
    ('s', 't'): 1
    ('t', '_'): 3
    ```

    - The most frequent pair is `('e', 'r')`.

    **Merge `('e', 'r')`**:
    ```
    'low _'      -> 2
    'low er _'   -> 1
    'n e w _'    -> 2
    'n e w er _' -> 1
    'b e t t er _' -> 2
    'b e s t _'  -> 1
    ```

    **New symbol pairs**:
    ```
    ('low', '_'): 3
    ('er', '_'): 3
    ('n', 'e'): 3
    ('e', 'w'): 3
    ('b', 'e'): 3
    ('e', 't'): 2
    ('t', 't'): 2
    ('s', 't'): 1
    ('t', '_'): 3
    ```

    - The most frequent pair is `('n', 'e')`.

    **Merge `('n', 'e')`**:
    ```
    'low _'      -> 2
    'low er _'   -> 1
    'ne w _'     -> 2
    'ne w er _'  -> 1
    'b e t t er _' -> 2
    'b e s t _'  -> 1
    ```

### Final Tokenized Vocabulary:
- `'low _'`      -> 2
- `'low er _'`   -> 1
- `'ne w _'`     -> 2
- `'ne w er _'`  -> 1
- `'b e t t er _' -> 2
- `'b e s t _'`  -> 1

### Python Implementation of BPE

In [14]:
from collections import defaultdict

def get_pairs(word):
    """Returns a set of symbol pairs in a word."""
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs

def print_dic(d:dict, rowlen = 6):
    """Print a dictionary in multiple lines"""
    r = 1
    for k,v in d.items():
        if(r%rowlen == 0):
            print()

        print(k, ":", v, sep='', end=', ')
        r = r+1
    print()

def merge_pair(pair, vocab):
    """Merge all occurrences of the most frequent pair in the vocabulary."""
    new_vocab = {}
    bigram = ' '.join(pair)
    replacement = ''.join(pair)
    for word in vocab:
        new_word = word.replace(bigram, replacement)
        new_vocab[new_word] = vocab[word]
    return new_vocab

def bpe_tokenizer(vocab, num_merges):
    """Main BPE tokenization function."""
    for _ in range(num_merges):
        # Get the frequency of all pairs in the vocabulary
        pair_counts = defaultdict(int)
        for word, freq in vocab.items():
            pairs = get_pairs(word.split())
            for pair in pairs:
                pair_counts[pair] += freq
        
        if not pair_counts:
            break

        # Find the most frequent pair
        best_pair = max(pair_counts, key=pair_counts.get)
        print("\nPairs:", end=' ')
        print_dic(pair_counts)
        print("\nMerge:", f'{best_pair}: {pair_counts[best_pair]}')
        
        # Merge the most frequent pair in the vocabulary
        print('Old:', vocab)
        vocab = merge_pair(best_pair, vocab)
        print('New:', vocab)

    return vocab

# Toy example: Vocabulary with word frequencies
vocab = {
    'l o w _': 2,
    'l o w e r _': 1,
    'n e w _': 2,
    'n e w e r _': 1,
    'b e t t e r _': 2,
    'b e s t _': 1
}

# Perform 4 merges
num_merges = 4
final_vocab = bpe_tokenizer(vocab, num_merges)


# Display the final tokenized vocab
print("\nFinal vocabulary:")
print(final_vocab)


Pairs: ('l', 'o'):3, ('o', 'w'):3, ('w', '_'):4, ('r', '_'):4, ('e', 'r'):4, 
('w', 'e'):2, ('n', 'e'):3, ('e', 'w'):3, ('b', 'e'):3, ('t', 't'):2, ('t', 'e'):2, 
('e', 't'):2, ('t', '_'):1, ('e', 's'):1, ('s', 't'):1, 

Merge: ('w', '_'): 4
Old: {'l o w _': 2, 'l o w e r _': 1, 'n e w _': 2, 'n e w e r _': 1, 'b e t t e r _': 2, 'b e s t _': 1}
New: {'l o w_': 2, 'l o w e r _': 1, 'n e w_': 2, 'n e w e r _': 1, 'b e t t e r _': 2, 'b e s t _': 1}

Pairs: ('l', 'o'):3, ('o', 'w_'):2, ('r', '_'):4, ('e', 'r'):4, ('o', 'w'):1, 
('w', 'e'):2, ('e', 'w_'):2, ('n', 'e'):3, ('e', 'w'):1, ('b', 'e'):3, ('t', 't'):2, 
('t', 'e'):2, ('e', 't'):2, ('t', '_'):1, ('e', 's'):1, ('s', 't'):1, 

Merge: ('r', '_'): 4
Old: {'l o w_': 2, 'l o w e r _': 1, 'n e w_': 2, 'n e w e r _': 1, 'b e t t e r _': 2, 'b e s t _': 1}
New: {'l o w_': 2, 'l o w e r_': 1, 'n e w_': 2, 'n e w e r_': 1, 'b e t t e r_': 2, 'b e s t _': 1}

Pairs: ('l', 'o'):3, ('o', 'w_'):2, ('e', 'r_'):4, ('o', 'w'):1, ('w', 'e'):2, 
('

### Tokenization Algorithm

New inputs are tokenized closely following the BPE training process:

1. **Normalization**: The input text is normalized.
2. **Pre-tokenization**: The normalized text is split into words.
3. **Character Splitting**: Each word is split into individual characters.
4. **Applying Merge Rules**: The learned BPE merge rules are applied in sequence to the split characters.

### 🍎**Example**:


### A simplified BPE implementation from [HuggingFace](https://huggingface.co/learn/nlp-course/chapter6/5)

1. **Create a Corpus**: 
   - Example sentences are used to form a basic corpus:

In [ ]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

2. **Pre-tokenization**:
   - Use a pre-trained tokenizer (like GPT-2) to split the corpus into words.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

3. **Compute Word Frequencies**:
   - Calculate the frequency of each word in the corpus.

In [ ]:
from collections import defaultdict
word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
print(word_freqs)

4. **Compute Base Vocabulary**:
   - Extract and sort all unique characters (alphabet) in the corpus.

In [ ]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
print(alphabet)

   - Include special tokens like GPT-2’s `<|endoftext|>`:

In [ ]:
vocab = ["<|endoftext|>"] + alphabet.copy()

5. **Split Words into Characters**:
   - Initialize word splits by breaking words into individual characters:

In [ ]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

6. **Compute Pair Frequencies**:
   - Write a function to calculate the frequency of character pairs, which is essential for training the BPE merges:

In [ ]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

   - Example output for pair frequencies:

In [ ]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

- Find the most frequent pair

In [ ]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

- The first merge to learn is ('Ġ', 't') -> 'Ġt'
  - add 'Ġt' to the vocabulary

In [ ]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

- Apply that merge in the splits dictionary with another function

In [ ]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

# The result of the first merge
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

- Loop until we have learned all the merges rules
  - suppose a vocab size of 50:

In [ ]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

- 19 merge rules learned 
   - the initial vocabulary had a size of 31 — 30 characters in the alphabet, 
   - plus the special token

In [ ]:
print(len(merges), ":\n", merges)

- The vocabulary is composed of 
  - the special token, 
  - the initial alphabet, 
  - and all the results of the merges

In [ ]:
print(vocab)

- To tokenize a new text, 
  - we pre-tokenize it, split it, 
  - then apply all the merge rules learned

In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

- Try this on any text composed of characters in the alphabet:

In [ ]:
tokenize("This is natural language processing BPE algorithm.")

## Unigram Algorithm

- Used in SentencePiece, applied in models like AlBERT, T5, mBART, Big Bird, and XLNet.
- Unlike BPE and WordPiece, Unigram starts with a large vocabulary and prunes it down.


### **Training Algorithm**

- Begins with a large vocabulary
  - Options for building initial vocabulary:
    - Use the most common substrings in pre-tokenized words.
    - Apply BPE with a large vocabulary size.
- Vocabulary Pruning
  - Computes loss for the corpus using the current vocabulary.
  - Calculates the impact on loss if each symbol is removed.
  - Removes the least impactful symbols until the desired vocabulary size is achieved.
    - Base characters are never removed to ensure any word can be tokenized.

---

### **Tokenization Process**
- **Concept**:
  - Unigram is a language model where each token is independent.
  - Probability of a token = (Frequency of the token) / (Sum of all frequencies in the vocabulary).
- 🍎 **Example**: 
  - Given the corpus: 
    - ("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)
  - Initial vocabulary of all strict substrings: 
    - ["h", "u", "g", "hu", "ug", "p", "pu", "n", "un", "b", "bu", "s", "hug", "gs", "ugs"]
  - Subword frequencies:
    - ("h", 15) ("u", 36) ("g", 20) ("hu", 15) ("ug", 20) ("p", 17) ("pu", 17) ("n", 16)
    - ("un", 16) ("b", 4) ("bu", 4) ("s", 5) ("hug", 15) ("gs", 5) ("ugs", 5)
  - ∴ the sum of all frequencies is 210
    - the probability of the subword "ug" is thus 20/210
    - the probability of the subword "ugs" is thus 5/210

---

### **Finding Optimal Tokenization**
- **Probability Computation**:
  - Given a word, `segment it into tokens in all the possible segmentations` 
    - compute the probability p of each according to the Unigram model
    - all tokens are considered independent, this probability is just the product of the probability of each token
- Tokenization with the highest probability is selected.
- 🍎 **Example**
  - Given word "pug", all its possible segmentations
    - p(['p','u','g']) = p('p') × p('u') × p('g') = (5/210)×(36/210)×(20/210) = 0.000389
    - p(['p','ug']) = p('p') × p('ug') = (5/210)×(20/210) = 0.0022676
    - p(['pu','g']) = p('pu') × p('g') = (5/210)×(20/210) = 0.0022676
  - Tokenizations with the least tokens possible will have the highest probability
    - Segmenations in a tie choose the first encountered

---

### **Viterbi Algorithm**
- **Purpose**:
  - Finds the most probable segmentation for a given word.
- **Process**:
  - Constructs a graph of possible segmentations.
  - Determines the best path through the graph, ending in the highest score.
- 🍎 **Example**
  - Given the previous vocabulary, for each position in `unhug`, the subwords with the `best scores ending there` are the following
    - Character 0 (u): "u" (score 0.171429)
    - Character 1 (n): "un" (score 0.076191)
    - Character 2 (h): "un" "h" (score 0.005442)
    - Character 3 (u): "un" "hu" (score 0.005442)
    - Character 4 (g): "un" "hug" (score 0.005442)
  - ∴  "unhug" would be tokenized as ["un", "hug"].

---

### **Back to Training**
- **Loss Computation**:
  - `Loss` is the negative log likelihood of tokenization scores across the corpus.
    - $\displaystyle 𝓁(corpus) = ∑_{w∈corpus} -f(w)\log(p(w))$
- 🍎 **Example**: Loss computation for the previous corpus `("hug", "pug", "pun", "bun", "hugs")`.
  - "hug": ["hug"] (score 0.071428)
  - "pug": ["pu", "g"] (score 0.007710)
  - "pun": ["pu", "n"] (score 0.006168)
  - "bun": ["bu", "n"] (score 0.001451)
  - "hugs": ["hug", "s"] (score 0.001701)
- ∴ loss = 10 * (-log(0.071428)) + 5 * (-log(0.007710)) + 12 * (-log(0.006168)) + 4 * (-log(0.001451)) + 5 * (-log(0.001701)) = 169.8

## **Implementing Unigram**

- **Objective**: Implement the Unigram algorithm in code.
- **Note**: This is a basic implementation, not optimized for efficiency.



### **Corpus for Unigram Implementation**
- **Tokenizer**: [`xlnet-base-cased`](https://arxiv.org/pdf/1906.08237)
- **Example Corpus**:

In [1]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

### **Counting Word Frequencies**

- **Count the occurrences of each word**:

In [2]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'▁This': 3,
             '▁is': 2,
             '▁the': 1,
             '▁Hugging': 1,
             '▁Face': 1,
             '▁Course.': 1,
             '▁chapter': 1,
             '▁about': 1,
             '▁tokenization.': 1,
             '▁section': 1,
             '▁shows': 1,
             '▁several': 1,
             '▁tokenizer': 1,
             '▁algorithms.': 1,
             '▁Hopefully,': 1,
             '▁you': 1,
             '▁will': 1,
             '▁be': 1,
             '▁able': 1,
             '▁to': 1,
             '▁understand': 1,
             '▁how': 1,
             '▁they': 1,
             '▁are': 1,
             '▁trained': 1,
             '▁and': 1,
             '▁generate': 1,
             '▁tokens.': 1})

### **Initializing Vocabulary**

- **Create frequency dictionaries for characters and subwords**:

In [13]:
char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

- **Sort subwords by frequency**:

In [14]:
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords[:10]

[('▁t', 7),
 ('is', 5),
 ('er', 5),
 ('▁a', 5),
 ('▁to', 4),
 ('to', 4),
 ('en', 4),
 ('▁T', 3),
 ('▁Th', 3),
 ('▁Thi', 3)]

### **Building the Initial Vocabulary**

- **Combine characters with top subwords to get an initial vocabulary of size 300**
- **Note**: SentencePiece uses Enhanced Suffix Array (ESA) for efficiency.

In [15]:
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}

### **Converting Frequencies to Probabilities**

- **Convert frequencies to probabilities using logarithms**
  - Because it’s more numerically stable to add logarithms than to multiply small numbers

In [16]:
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

### **Tokenizing Words with Viterbi Algorithm**

- **Main function to tokenize words**:

In [17]:
def encode_word(word, model):
    # computes the best segmentation of each substring of the word
    # store one dictionary per position in the word (from 0 to its total length), 
    # with two keys: 
    #   - the index of the start of the last token in the best segmentation, 
    #   - and the score of the best segmentation
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    # 1. goes over each start position
    for start_idx in range(len(word)):
        best_score_at_start = best_segmentations[start_idx]["score"]
        # 2. tries all substrings beginning at that start position
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score

### **Testing the Tokenizer**

- **Test the tokenizer with sample words**:

In [18]:
print(encode_word("Hopefully", model))
print(encode_word("This", model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 41.5157494601402)
(['This'], 6.288267030694535)


### **Computing Model Loss**

- **Function to compute the model's loss on the corpus**:

In [9]:
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

compute_loss(model)

413.10377642940875

### **Computing Scores for Tokens**

- **Function to compute scores by removing each token**:

In [19]:
import copy

def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores

- Try it on given tokens:

In [20]:
scores = compute_scores(model)
# "ll" is used in the tokenization of "Hopefully", 
# and removing it may leads to use the token "l" twice instead, 
# so it will have a positive loss
print(scores["ll"]) 

# "his" is only used inside the word "This", 
# which is tokenized as itself, 
# so it will have a zero loss
print(scores["his"])

6.376412403623874
0.0


### **Pruning the Vocabulary**

- **Loop until vocabulary size is reduced to the desired size**:

In [21]:
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])

    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

### **Final Tokenization Function**

- **Tokenize text using the trained model**:

In [12]:
def tokenize(text, model):
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in words_with_offsets]
    encoded_words = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_words, [])

tokenize("This is the Hugging Face course.", model)

['▁This',
 '▁is',
 '▁the',
 '▁Hugging',
 '▁Face',
 '▁',
 'c',
 'ou',
 'r',
 's',
 'e',
 '.']

### 🏃 **Exercise on Token Removal**
- **Impact on Loss**:
  - Demonstrates how removing tokens affects the loss.
  - Example: Removing "pu" vs. removing "hug".

### Explanation of the Minimum Edit Distance Algorithm

The **minimum edit distance** algorithm calculates the number of edits (insertions, deletions, or substitutions) required to transform one string into another. The algorithm is commonly implemented using **dynamic programming**. The edit distance between two strings can be used to measure how similar they are, with a smaller distance indicating higher similarity.

#### Types of operations:
- **Insertion**: Insert a character into the string.
- **Deletion**: Remove a character from the string.
- **Substitution**: Replace one character with another.

### Algorithm Steps:

1. Create a matrix (or table) `dp` of size `(m+1) x (n+1)`, where `m` is the length of the first string `A` and `n` is the length of the second string `B`.
2. Initialize the first row and column of the matrix to represent the edit distance for transforming a string into an empty string.
3. For each character pair in both strings, fill the matrix according to the following rules:
   - If the characters are equal, no operation is needed, so use the previous diagonal value.
   - If the characters are different, choose the minimum of:
     - Inserting a character,
     - Deleting a character,
     - Substituting a character,
   and add 1 to the result.
4. The value at `dp[m][n]` will be the minimum edit distance between the two strings.

### Example:

Suppose we want to find the minimum edit distance between the words `"intention"` and `"execution"`.

### Step-by-step Table Calculation

|     |    | e  | x  | e  | c  | u  | t  | i  | o  | n  |
|-----|----|----|----|----|----|----|----|----|----|----|
|     |  0 | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  |
| i   |  1 | 1  | 2  | 3  | 4  | 5  | 5  | 6  | 7  | 8  |
| n   |  2 | 2  | 2  | 3  | 4  | 5  | 6  | 6  | 7  | 7  |
| t   |  3 | 3  | 3  | 3  | 4  | 5  | 5  | 6  | 7  | 8  |
| e   |  4 | 3  | 4  | 3  | 4  | 5  | 6  | 6  | 7  | 8  |
| n   |  5 | 4  | 4  | 4  | 4  | 5  | 6  | 7  | 7  | 7  |
| t   |  6 | 5  | 5  | 5  | 5  | 5  | 6  | 7  | 8  | 8  |
| i   |  7 | 6  | 6  | 6  | 6  | 6  | 6  | 6  | 7  | 8  |
| o   |  8 | 7  | 7  | 7  | 7  | 7  | 7  | 7  | 6  | 7  |
| n   |  9 | 8  | 8  | 8  | 8  | 8  | 8  | 8  | 7  | 6  |

Here:
- Insertion and deletion are represented by increments in row and column values.
- Substitution occurs when the characters do not match.

The final minimum edit distance between `"intention"` and `"execution"` is **5** (found in the bottom-right corner).

### Python Implementation

```python
def minimum_edit_distance(A, B):
    m, n = len(A), len(B)
    
    # Create a (m+1) x (n+1) DP matrix
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Initialize the first row and column
    for i in range(m + 1):
        dp[i][0] = i  # Cost of converting A[0..i] to an empty string
    for j in range(n + 1):
        dp[0][j] = j  # Cost of converting an empty string to B[0..j]

    # Fill the DP table
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if A[i - 1] == B[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # Characters match, no cost
            else:
                dp[i][j] = min(dp[i - 1][j] + 1,   # Deletion
                               dp[i][j - 1] + 1,   # Insertion
                               dp[i - 1][j - 1] + 1)  # Substitution

    # The minimum edit distance is in the bottom-right corner
    return dp[m][n], dp

# Test example
A = "intention"
B = "execution"
distance, dp_table = minimum_edit_distance(A, B)

# Print results
print(f"Minimum Edit Distance between '{A}' and '{B}' is {distance}")
print("DP Table:")
for row in dp_table:
    print(row)
```

### Output:

```
Minimum Edit Distance between 'intention' and 'execution' is 5
DP Table:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 1, 2, 3, 4, 5, 6, 6, 7, 8]
[2, 2, 2, 3, 4, 5, 6, 6, 7, 7]
[3, 3, 3, 3, 4, 5, 5, 6, 7, 8]
[4, 3, 4, 3, 4, 5, 6, 6, 7, 8]
[5, 4, 4, 4, 4, 5, 6, 7, 7, 7]
[6, 5, 5, 5, 5, 5, 6, 7, 8, 8]
[7, 6, 6, 6, 6, 6, 6, 6, 7, 8]
[8, 7, 7, 7, 7, 7, 7, 7, 6, 7]
[9, 8, 8, 8, 8, 8, 8, 8, 7, 6]
```

In this Python implementation, the minimum edit distance between `"intention"` and `"execution"` is calculated as **5**. The table `dp_table` shows the dynamic programming table used to compute the result.